### Connect to the workspace

In [1]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with mlw-test


### Let's take a look at models registered in the workspace

In [2]:
from azureml.core import Model

for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

diabetes-model version: 7
	 Algorithm : Decision Tree Classifier
	 Training context : Pipeline
	 AUC : 0.8832546157718848
	 Accuracy : 0.898


diabetes-model version: 6
	 Algorithm : Logistic Regression
	 Training context : Tabular diabetes data asset
	 AUC : 0.8568650620553335
	 Accuracy : 0.7893333333333333
	 Regularization Rate : 0.1


diabetes-model version: 5
	 Algorithm : Decision Tree Classifier
	 Training context : Pipeline
	 AUC : 0.88375696004516
	 Accuracy : 0.8986666666666666


diabetes-model version: 4
	 Algorithm : Logistic Regression
	 Training context : Tabular diabetes data asset
	 AUC : 0.8568650620553335
	 Accuracy : 0.7893333333333333
	 Regularization Rate : 0.1


diabetes-model version: 3
	 Training context : Tabular diabetes data asset
	 AUC : 0.8568650620553335
	 Accuracy : 0.7893333333333333
	 Regularization Rate : 0.1


diabetes-model version: 2
	 Training context : Tabular diabetes data asset
	 AUC : 0.8568650620553335
	 Accuracy : 0.7893333333333333
	 Regular

### Get the model that we want to deploy. By default, if we specify a model name, the latest version will be returned.

In [3]:
model = ws.models['diabetes-model']
print(model.name, 'version', model.version)

diabetes-model version 7


### As we are going to create a web service to host this model, this will require some code and configuration files; so let's create a folder for those.

In [4]:
import os

# Create a folder for the deployment files
deployment_folder = './diabetes-service'
os.makedirs(deployment_folder, exist_ok=True)
print(deployment_folder, 'folder created.')

# Set path for scoring script
script_file = 'score_diabetes.py'
script_path = os.path.join(deployment_folder,script_file)

./diabetes-service folder created.


The web service where we deploy the model will need some Python code to load the input data, get the model from the workspace, and generate and return predictions. We'll save this code in an entry script (often called a scoring script) that will be deployed to the web service.

The script consists of two functions:

**init:** This function is called when the service is initialized, and is generally used to load the model. Note that the scoring script uses the AZUREML_MODEL_DIR environment variable to determine the folder where the model is stored.

**run:** This function is called each time a client application submits new data, and is generally used to inference predictions from the model.

In [5]:
%%writefile $script_path
import json
import joblib
import numpy as np
import os

# Called when the service is loaded
def init():
    global model
    # Get the path to the deployed model file and load it
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'diabetes_model.pkl')
    model = joblib.load(model_path)

# Called when a request is received
def run(raw_data):
    # Get the input data as a numpy array
    data = np.array(json.loads(raw_data)['data'])
    # Get a prediction from the model
    predictions = model.predict(data)
    # Get the corresponding classname for each prediction (0 or 1)
    classnames = ['not-diabetic', 'diabetic']
    predicted_classes = []
    for prediction in predictions:
        predicted_classes.append(classnames[prediction])
    # Return the predictions as JSON
    return json.dumps(predicted_classes)

Overwriting ./diabetes-service/score_diabetes.py


The web service will be hosted in a container, and the container will need to install any required Python dependencies when it gets initialized. In this case, our scoring code requires scikit-learn and some Azure Machine Learning specific packages that are used by the scoring web service, so we'll create an environment that included these. Then we'll add that environment to an inference configuration along with the scoring script, and define a deployment configuration for the container in which the environment and script will be hosted.

We can then deploy the model as a service based on these configurations.

More Information: For more details about model deployment, and options for target execution environments, see the documentation.
Deployment will take some time as it first runs a process to create a container image, and then runs a process to create a web service based on the image. When deployment has completed successfully, you'll see a status of Healthy.

In [6]:
service_name = "diabetes-service"

In [7]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import LocalWebservice

# Configure the scoring environment
service_env = Environment.get(workspace=ws, name="AzureML-sklearn-0.24.1-ubuntu18.04-py37-cpu-inference")
service_env.inferencing_stack_version="latest"

In [8]:
inference_config = InferenceConfig(source_directory=deployment_folder,
                                   entry_script=script_file,
                                   environment=service_env)

In [9]:
deploy_to_container_instance = True

### Deploy to an Azure Container Instance

In [10]:
if deploy_to_container_instance:
    print('Deploying model as Azure Container Image...')
    deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)
    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
    service.wait_for_deployment(True)
else:
    print('Deploying model as local web service...')
    deployment_config = LocalWebservice.deploy_configuration(port=8890)
    service = Model.deploy(ws, service_name, [model], inference_config, deployment_config)

print(service.state)

Deploying model as Azure Container Image...
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-01-31 18:40:49+00:00 Creating Container Registry if not exists.
2023-01-31 18:40:49+00:00 Registering the environment.
2023-01-31 18:40:50+00:00 Use the existing image.
2023-01-31 18:40:50+00:00 Generating deployment configuration.
2023-01-31 18:40:51+00:00 Submitting deployment to compute.
2023-01-31 18:40:57+00:00 Checking the status of deployment diabetes-service..
2023-01-31 18:43:12+00:00 Checking the status of inference endpoint diabetes-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


### Or deploy to a local web service for troubleshooting

In [11]:
print(service.get_logs())

# If you need to make a change and redeploy, you may need to delete unhealthy service using the following code:
#service.delete()

2023-01-31T18:43:04,949912639+00:00 - gunicorn/run 
2023-01-31T18:43:04,954112033+00:00 | gunicorn/run | 
2023-01-31T18:43:04,960681824+00:00 | gunicorn/run | ###############################################
2023-01-31T18:43:04,962516622+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-01-31T18:43:04,968928413+00:00 | gunicorn/run | ###############################################
2023-01-31T18:43:04,970488811+00:00 | gunicorn/run | 
2023-01-31T18:43:04,984405892+00:00 | gunicorn/run | 
2023-01-31T18:43:05,003823566+00:00 | gunicorn/run | AzureML image information: sklearn-0.24.1-ubuntu18.04-py37-cpu-inference:20221024.v1
2023-01-31T18:43:05,005948463+00:00 | gunicorn/run | 
2023-01-31T18:43:05,015695150+00:00 | gunicorn/run | 
2023-01-31T18:43:05,017593447+00:00 | gunicorn/run | PATH environment variable: /opt/miniconda/envs/amlenv/bin:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
2023-01-31T18:43:05,025597936+00:00 | gunicorn/run | PYT

In [12]:
for webservice_name in ws.webservices:
    print(webservice_name)

diabetes-service


In [13]:
import json

x_new = [[2,180,74,24,21,23.9091702,1.488172308,22]]
print ('Patient: {}'.format(x_new[0]))

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
predicted_classes = json.loads(predictions)
print(predicted_classes[0])

Patient: [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22]
diabetic


In [14]:
import json

# This time our input is an array of two feature arrays
x_new = [[2,180,74,24,21,23.9091702,1.488172308,22],
         [0,148,58,11,179,39.19207553,0.160829008,45]]

# Convert the array or arrays to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data
predictions = service.run(input_data = input_json)

# Get the predicted classes.
predicted_classes = json.loads(predictions)
   
for i in range(len(x_new)):
    print ("Patient {}".format(x_new[i]), predicted_classes[i] )

Patient [2, 180, 74, 24, 21, 23.9091702, 1.488172308, 22] diabetic
Patient [0, 148, 58, 11, 179, 39.19207553, 0.160829008, 45] diabetic


In [15]:
endpoint = service.scoring_uri
print(endpoint)

http://4bd72f7e-dd71-452c-88ce-89966b25664f.westeurope.azurecontainer.io/score


In [16]:
#service.delete()
#print ('Service deleted.')